pytorch会自动在batch维度上扩充

In [1]:
from models import ast_models, models
from Dataset.dataset import load_LJSpeech

import torch
import torch.nn.functional as F
import torch.nn as nn
import torchaudio

import numpy as np
import time

In [1]:
from Dataset.mel2samp import Mel2Samp
from Dataset.preparation import data_prepar, data_prepar2, data_prepar_spk, data_prepar_joint, data_prepar_joint_spk

In [2]:
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
D:\anaconda\envs\pytorch19\lib\site-packages\huggingface_hub\utils\_deprecation.py:39: FutureWarning: Pass library_name=False as keyword args. From version 0.7 passing these as positional arguments will result in an error
  warnings.warn(


OSError: [WinError 1314] 客户端没有所需的特权。: 'C:\\Users\\LYC\\.cache\\huggingface\\hub\\e2383704e6d91a969376e2b5690a049debc911cdad63bb4c1bf51301ff4359c5.e4712caf19ea894d30be5fea900d875025eb98ae7287543f0a39d166bf595658' -> 'pretrained_models\\EncoderClassifier--8446833932736073137\\hyperparams.yaml'

### Datasets

In [2]:
# train_loader = load_LJSpeech('Dataset/pitch_scaling.txt')
train_loader = load_LJSpeech('Dataset/pitch_scaling_joint2.txt', speaker_emb=True, joint_factor=True)

In [3]:
len(train_loader)

7907

In [2]:
input_tdim = 500
ast_mdl = ast_models.ASTModel(label_dim=1,
                    fshape=128, tshape=2, fstride=128, tstride=1,
                    input_fdim=128, input_tdim=input_tdim, model_size='base',
                    pretrain_stage=False, load_pretrained_mdl_path='./save_model/SSAST-Base-Frame-400.pth')
                      

ast_mdl = ast_mdl.cuda()

now load a SSL pretrained models from ./save_model/SSAST-Base-Frame-400.pth
pretraining patch split stride: frequency=128, time=2
pretraining patch shape: frequency=128, time=2
pretraining patch array dimension: frequency=1, time=512
pretraining number of patches=512
fine-tuning patch split stride: frequncey=128, time=1
fine-tuning number of patches=499


D:\anaconda\envs\pytorch19\lib\site-packages\torch\nn\functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


In [5]:
estimator = models.estimator()
estimator = estimator.cuda()

In [3]:
state_dict = torch.load('./save_model/ft90.pth')
ast_mdl.load_state_dict(state_dict, strict=False)

<All keys matched successfully>

In [3]:
state_dict = torch.load('./save_model/ft_joint3_4.pth')
ast_mdl.load_state_dict(state_dict, strict=False)

<All keys matched successfully>

In [8]:
learning_rate = 1e-04
loss_fn = nn.MSELoss()
# opt1 = torch.optim.Adam(ast_mdl.parameters(), lr = 0.1*learning_rate, betas=[0.9, 0.999])
opt2 = torch.optim.Adam(estimator.parameters(), lr = learning_rate, betas=[0.9, 0.999])

NameError: name 'estimator' is not defined

In [7]:
def train_joint(num_eps):
    for eps in range(num_eps):
        i = 0
        ast_mdl.train()
        since = time.time()
        loss_train, loss_valid = [], []
        #training phase
        for i, (fbank, label) in enumerate(train_loader): #fbank: [B, D, T]; 
            fbank = fbank.cuda()
            label = label.cuda().unsqueeze(-1)
            # print(label.shape)
            
            # #warming up
            # if global_step <= 1000 and global_step % 50 == 0 and args.warmup == True:
            #     for group_id, param_group in enumerate(optimizer.param_groups):
            #         warm_lr = (global_step / 1000) * lr_list[group_id]
            #         param_group['lr'] = warm_lr
            #         print('warm-up learning rate is {:f}'.format(param_group['lr']))
                    
                    
            pred_alpha = ast_mdl(fbank, task='ft_cls')
            
            loss = loss_fn(pred_alpha.float(), label.float())
            loss_train.append(loss)
            
            opt.zero_grad()
            loss.backward()
            opt.step()

            if i % 32 == 0:
                print ('.', end='')
            i += 1
        
        
#         #valid phase
#         ast_mdl.eval()
#         for i, (fbank, label) in enumerate(valid_loader): #fbank: [B, D, T];    
#             fbank = fbank.cuda()
#             label = label.cuda().unsqueeze(-1)
              
#             pred_alpha = ast_mdl(fbank, task='ft_cls') 
#             loss = loss_fn(pred_alpha.float(), label.float())
#             loss_valid.append(loss)

#             if i % 32 == 0:
#                 print ('.', end='')
#             i += 1
            
        
        time_elapsed = time.time() - since    
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
        if eps % 1 == 0:
            print('loss_train:', sum(loss_train)/len(loss_train)) #, 'loss_train:', sum(loss_valid)/len(loss_valid))

        if (eps + 1) % 5 == 0:
            torch.save(ast_mdl.state_dict(), 'save_model/ft_joint3_'+str(eps)+'.pth')
            print ('Saving checkpoint for epoch {} at {}'.format(eps+1,
                                                         'save_model/ft_joint3_'+str(eps)+'.pth'))

In [7]:
def train_emb(num_eps):
    for eps in range(num_eps):
        i = 0
        ast_mdl.train()
        estimator.train()
        since = time.time()
        loss_train, loss_valid = [], []
        #training phase
        for i, (fbank, fbank_org, label) in enumerate(train_loader): #fbank: [B, D, T]; 
            fbank = fbank.cuda()
            fbank_org = fbank_org.cuda()
            label = label.cuda().unsqueeze(-1)
            # print(label.shape)
            
            # #warming up
            # if global_step <= 1000 and global_step % 50 == 0 and args.warmup == True:
            #     for group_id, param_group in enumerate(optimizer.param_groups):
            #         warm_lr = (global_step / 1000) * lr_list[group_id]
            #         param_group['lr'] = warm_lr
            #         print('warm-up learning rate is {:f}'.format(param_group['lr']))
                    
                    
            emb_scaled = ast_mdl(fbank, task='ft_emb').detach()
            emb_org = ast_mdl(fbank_org, task='ft_emb').detach()
            pred_alpha = estimator(emb_scaled, emb_org)
            
            loss = loss_fn(pred_alpha.float(), label.float())
            loss_train.append(loss)
            
            # opt1.zero_grad(), 
            opt2.zero_grad()
            loss.backward()
            # opt1.step(), 
            opt2.step()

            if i % 32 == 0:
                print ('.', end='')
            i += 1
        
        
#         #valid phase
#         ast_mdl.eval()
#         for i, (fbank, label) in enumerate(valid_loader): #fbank: [B, D, T];    
#             fbank = fbank.cuda()
#             label = label.cuda().unsqueeze(-1)
              
#             pred_alpha = ast_mdl(fbank, task='ft_cls') 
#             loss = loss_fn(pred_alpha.float(), label.float())
#             loss_valid.append(loss)

#             if i % 32 == 0:
#                 print ('.', end='')
#             i += 1
            
        
        time_elapsed = time.time() - since    
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
        if eps % 1 == 0:
            print('loss_train:', sum(loss_train)/len(loss_train)) #, 'loss_train:', sum(loss_valid)/len(loss_valid))

        if (eps + 1) % 5 == 0:
            torch.save(ast_mdl.state_dict(), 'save_model/ft_estimator_'+str(eps+1)+'.pth')
            print ('Saving checkpoint for epoch {} at {}'.format(eps+1,
                                                         'save_model/ft_estimator_'+str(eps+1)+'.pth'))

In [8]:
def main():
    train_emb(100)

In [9]:
if __name__=="__main__":
    main() 

........................................................................................................................................................................................................................................................Training complete in 47m 59s
loss_train: tensor(5.3310, device='cuda:0', grad_fn=<DivBackward0>)
........................................................................................................................................................................................................................................................Training complete in 47m 4s
loss_train: tensor(4.4998, device='cuda:0', grad_fn=<DivBackward0>)
........................................................................................................................................................................................................................................................Training complete in 46m 53s
loss_train: tensor(4.3928, device=

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001E3D6D155E0>
Traceback (most recent call last):
  File "D:\anaconda\envs\pytorch19\lib\site-packages\torch\utils\data\dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "D:\anaconda\envs\pytorch19\lib\site-packages\torch\utils\data\dataloader.py", line 1301, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "D:\anaconda\envs\pytorch19\lib\multiprocessing\process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "D:\anaconda\envs\pytorch19\lib\multiprocessing\popen_spawn_win32.py", line 108, in wait
    res = _winapi.WaitForSingleObject(int(self._handle), msecs)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [8]:
train_emb(100)

...............

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\anaconda\envs\pytorch19\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\LYC\AppData\Local\Temp/ipykernel_14952/3434716651.py", line 1, in <module>
    train_emb(100)
  File "C:\Users\LYC\AppData\Local\Temp/ipykernel_14952/365283608.py", line 9, in train_emb
    for i, (fbank, fbank_org, label) in enumerate(train_loader): #fbank: [B, D, T];
  File "D:\anaconda\envs\pytorch19\lib\site-packages\torch\utils\data\dataloader.py", line 521, in __next__
    data = self._next_data()
  File "D:\anaconda\envs\pytorch19\lib\site-packages\torch\utils\data\dataloader.py", line 1186, in _next_data
    idx, data = self._get_data()
  File "D:\anaconda\envs\pytorch19\lib\site-packages\torch\utils\data\dataloader.py", line 1152, in _get_data
    success, data = self._try_get_data()
  File "D:\anaconda\envs\pytorch19\lib\site-packages\torch\utils\data\dataloader.

TypeError: object of type 'NoneType' has no len()

In [7]:
def train(num_eps):
    for eps in range(num_eps):
        i = 0
        ast_mdl.train()
        since = time.time()
        for i, (fbank, label_freq, label_time) in enumerate(train_loader): #fbank: [B, D, T]; 
            fbank = fbank.cuda()
            label_freq = label_freq.cuda()
            label_time = label_time.cuda()
            
            # #warming up
            # if global_step <= 1000 and global_step % 50 == 0 and args.warmup == True:
            #     for group_id, param_group in enumerate(optimizer.param_groups):
            #         warm_lr = (global_step / 1000) * lr_list[group_id]
            #         param_group['lr'] = warm_lr
            #         print('warm-up learning rate is {:f}'.format(param_group['lr']))
                    
                    
            pred_freq, pred_time = ast_mdl(fbank, task='ft_cls2')
            
            loss_freq = loss_fn(pred_freq.float(), label_freq.float())
            loss_time = loss_fn(pred_time.float(), label_time.float())
            loss = 2*loss_freq + 1*loss_time #set two hyperparemeters
            opt.zero_grad()
            loss.backward()
            opt.step()

            if i % 32 == 0:
                print ('.', end='')
            i += 1
            
        time_elapsed = time.time() - since    
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
        if eps % 1 == 0:
            print('loss:', loss, '\n' 'loss_freq:', loss_freq, '\n' 'loss_time:',loss_time)

        if (eps + 1) % 5 == 0:
            torch.save(ast_mdl.state_dict(), 'save_model/ft_multispk_'+str(eps)+'.pth')
            print ('Saving checkpoint for epoch {} at {}'.format(eps+1,
                                                         'save_model/ft_multispk_'+str(eps)+'.pth'))

In [8]:
def train_spk(num_eps):
    for eps in range(num_eps):
        i = 0
        ast_mdl.train()
        for i, (fbank, embedding, label_freq, label_time) in enumerate(train_loader): #fbank: [B, D, T]; 
            
            fbank = fbank.cuda() #确保embedding的shape:[B, 192]
            embedding = embedding.cuda()
            label_freq = label_freq.cuda()
            label_time = label_time.cuda()
            
            # #warming up
            # if global_step <= 1000 and global_step % 50 == 0 and args.warmup == True:
            #     for group_id, param_group in enumerate(optimizer.param_groups):
            #         warm_lr = (global_step / 1000) * lr_list[group_id]
            #         param_group['lr'] = warm_lr
            #         print('warm-up learning rate is {:f}'.format(param_group['lr']))                 
                    
            pred_freq, pred_time = ast_mdl(fbank, x_vector=embedding, task='ft_spk') #
            
            loss_freq = loss_fn(pred_freq.float(), label_freq.float())
            loss_time = loss_fn(pred_time.float(), label_time.float())
            loss = 1*loss_freq + 1*loss_time #set two hyperparemeters
            opt.zero_grad()
            loss.backward()
            opt.step()

            if i % 32 == 0:
                print ('.', end='')
            i += 1
            
        if eps % 1 == 0:
            print('loss:', loss, '\n' 'loss_freq:', loss_freq, '\n' 'loss_time:',loss_time)


        if (eps + 1) % 10 == 0:
            torch.save(ast_mdl.state_dict(), 'save_model/ft'+str(eps)+'.pth')
            print ('Saving checkpoint for epoch {} at {}'.format(eps+1,
                                                         ckpt_save_path))

In [9]:
eps=90

In [10]:
torch.save(ast_mdl.state_dict(), 'save_model/ft'+str(eps)+'.pth')

In [9]:
torch.save(ast_mdl.state_dict(), 'save_model/ft_multispk_'+str(eps)+'.pth')
print ('Saving checkpoint for epoch {} at {}'.format(eps+1, 'save_model/ft_multispk_'+str(eps)+'.pth'))

Saving checkpoint for epoch 11 at save_model/ft_multispk_10.pth


#### Valid

In [4]:
def get_mel(audio, sr):
    audio_norm = audio / 32768.0
    audio_norm = audio_norm.unsqueeze(0)
    audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    # melspec = self.stft(sr).mel_spectrogram(audio_norm)
    melspec = torchaudio.compliance.kaldi.fbank(audio, htk_compat=True, sample_frequency=sr, use_energy=False,
                                                    window_type='hanning', num_mel_bins=128, dither=0.0, frame_shift=10)
         
    melspec = torch.squeeze(melspec, 0)
    # melspec = torch.transpose(melspec, 0, 1)
        
    #cut and pad
    target_length = 500
    n_frames = melspec.shape[0]
    p = target_length - n_frames

    if p > 0:
        m = torch.nn.ZeroPad2d((0, 0, 0, p))
        melspec = m(melspec)
    elif p < 0:
        melspec = melspec[0:target_length, :]
   
    return melspec #(T, D)

In [10]:
audio, sr = torchaudio.load('E:/datasets/project3/AISHELL-3/test/wav/SSB0139/SSB01390019.wav')

In [30]:
audio, sr = torchaudio.load('E:/datasets/project3/LJSpeech-1.1/test/A2022612_f6.3.wav')

In [31]:
mel = get_mel(audio, sr).unsqueeze(0).cuda()

In [32]:
mel.shape

torch.Size([1, 500, 128])

In [33]:
alpha = ast_mdl(mel, task='ft_cls')

In [34]:
alpha

tensor([[4.8689]], device='cuda:0', grad_fn=<MulBackward0>)

In [40]:
pred_freq, pred_time = ast_mdl(mel, task='ft_cls2')

In [41]:
print(pred_freq, pred_time)

tensor([[3.7741]], device='cuda:0', grad_fn=<MulBackward0>) tensor([[0.8373]], device='cuda:0', grad_fn=<MulBackward0>)


In [23]:
import numpy as np
12*np.log2(0.92)+5

3.556469195387459

##### valid ASHELL-3

In [5]:
from Dataset.preparation import data_prepar, data_prepar2, data_prepar_spk, data_prepar_joint, data_prepar_joint_spk

In [6]:
test_files, test_labels = data_prepar_joint('Dataset/pitch_scaling_joint_t.txt')

In [7]:
loss_list, alpha_list = [], []
loss_fn = nn.MSELoss()
i = 0
for wav in test_files:
    audio, sr = torchaudio.load(wav)
    mel = get_mel(audio, sr).unsqueeze(0).cuda()
    pred_alpha = ast_mdl(mel, task='ft_cls').detach().squeeze(0)
    # label = torch.tensor(test_labels[i]).unsqueeze(0).cuda()
    # loss = loss_fn(pred_alpha.float(), label.float())
    # loss_list.append(loss)
    alpha_list.append(pred_alpha)
    # i+=1
# loss_list_mean = sum(loss_list)/len(loss_list)

In [8]:
alpha_list.shape

AttributeError: 'list' object has no attribute 'shape'

In [10]:
label = torch.tensor(test_labels).cuda()

In [19]:
label

tensor([ 0.0000,  1.5160, -4.1140,  ...,  6.2840, -5.6950,  0.0000],
       device='cuda:0')

In [9]:
alpha = torch.tensor(alpha_list).cuda()

In [24]:
alpha

tensor([ 0.5534,  0.5534, -4.0550,  ...,  0.2944, -5.9476, -5.9476],
       device='cuda:0')

In [19]:
err = (alpha - label).abs().mean()

In [20]:
err

tensor(0.1718, device='cuda:0')

In [16]:
e=0
for ele in err:
    if ele> 0.5:
        e+=1

In [17]:
er = e/len(err)

In [18]:
er

0.08114617940199335

In [18]:
len(label)

49545

In [13]:
loss_list_mean

tensor(9.8833, device='cuda:0')

#### add spk_emb

In [19]:
signal2, fs2 =torchaudio.load('E:/datasets/project3/LJSpeech-1.1/test/SSB06230043_f6_t1.2.wav')
embeddings2 = classifier.encode_batch(signal2)

In [20]:
embeddings2 = embeddings2.squeeze(0).cuda()

In [21]:
pred_freq, pred_time = ast_mdl(mel, x_vector=embeddings2, task='ft_spk')

In [22]:
print(pred_freq, pred_time)

tensor([[3.5455]], device='cuda:0', grad_fn=<MulBackward0>) tensor([[0.6319]], device='cuda:0', grad_fn=<MulBackward0>)
